In [1]:
import pandas as pd
from tqdm.notebook import tqdm
import plotly.graph_objects as go
import re

In [2]:
# !conda install plotly

In [3]:
for i in tqdm(range(100)):
    pass

  0%|          | 0/100 [00:00<?, ?it/s]

In [52]:
df_necessity = pd.read_csv('../datas/huge/생필품 농수축산물 가격 정보(2020년).csv', encoding='cp949')
df_population = pd.read_csv('../datas/huge/서울시 우리마을가게 상권분석서비스(상권-생활인구).csv', encoding='cp949')
df_sales = pd.read_csv('../datas/huge/서울시 우리마을가게 상권분석서비스(상권-추정매출)_2019.csv', encoding='cp949')

df_market_address = pd.read_csv('../datas/서울 전통시장 주소.csv', encoding='cp949')
# df_supermarket_address = pd.read_csv('../datas/서울 대형마트 주소.csv', encoding='cp949')

### 각 데이터 샘플

In [53]:
df_necessity.head()

,일련번호,시장/마트 번호,시장/마트 이름,품목 번호,품목 이름,실판매규격,가격(원),년도-월,비고,시장유형 구분(시장/마트) 코드,시장유형 구분(시장/마트) 이름,자치구 코드,자치구 이름,점검일자
0,1621162,1,통인시장,309,양파(1.5kg망),1kg,2000,2020-12,"국내산,무안",1,전통시장,110000,종로구,2020-12-21
1,1628949,228,방배종합시장,277,호박,1개268g,1500,2020-12,국내산/인큐베이터,1,전통시장,650000,서초구,2020-12-21
2,1626799,51,NC백화점 불광점,18,닭고기,1마리(1kg),8990,2020-12,올품 토종,2,대형마트,380000,은평구,2020-12-21
3,1630920,223,홈플러스 목동점,278,쇠고기,600g,35982,2020-12,국내산(음성)농협,2,대형마트,470000,양천구,2020-12-21
4,1622882,1,통인시장,309,양파(1.5kg망),1kg,2000,2020-12,"국내산,무안",1,전통시장,110000,종로구,2020-12-21


In [54]:
df_population.head()

,기준 년코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,총_생활인구_수,남성_생활인구_수,여성_생활인구_수,연령대_10_생활인구_수,...,여성연령대_60_이상_토요일시간대_3_생활인구_수,여성연령대_60_이상_토요일시간대_4_생활인구_수,여성연령대_60_이상_토요일시간대_5_생활인구_수,여성연령대_60_이상_토요일시간대_6_생활인구_수,여성연령대_60_이상_일요일시간대_1_생활인구_수,여성연령대_60_이상_일요일시간대_2_생활인구_수,여성연령대_60_이상_일요일시간대_3_생활인구_수,여성연령대_60_이상_일요일시간대_4_생활인구_수,여성연령대_60_이상_일요일시간대_5_생활인구_수,여성연령대_60_이상_일요일시간대_6_생활인구_수
0,2020,4,U,관광특구,1001496,강남 마이스 관광특구,90836,46313,44523,4351,...,142,140,123,39,48,99,124,123,104,35
1,2020,4,U,관광특구,1001495,잠실 관광특구,2956754,1445027,1511727,248574,...,5398,5785,7033,3989,7009,6350,5041,5460,6552,3868
2,2020,4,U,관광특구,1001494,종로?청계 관광특구,3738997,2060445,1678551,96484,...,10547,10296,7883,3577,5160,6020,6495,6618,6352,3992
3,2020,4,U,관광특구,1001493,동대문패션타운 관광특구,3058361,1494579,1563782,99823,...,5674,5149,4481,2629,4734,4903,4273,4182,5044,4354
4,2020,4,U,관광특구,1001492,명동 남대문 북창동 다동 무교동 관광특구,3569497,1853999,1715498,81863,...,9552,8998,4532,1217,1672,3462,4677,4407,3001,1164


In [55]:
df_sales.head()

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,분기당_매출_금액,분기당_매출_건수,...,시간대_건수~24_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수,점포수
0,2019,1,A,골목상권,1000985,암사길,CS200003,예술학원,20061598,125,...,0,93,32,0,0,32,93,0,0,3
1,2019,1,A,골목상권,1000267,돌곶이로8길,CS200037,노래방,510636,21,...,8,4,17,0,0,0,9,4,8,1
2,2019,1,R,전통시장,1001445,영도시장,CS300027,섬유제품,4038928,33,...,0,17,0,0,0,17,0,0,0,2
3,2019,1,A,골목상권,1000789,난곡로24길,CS300017,시계및귀금속,5085873,10,...,0,5,5,0,0,0,5,0,5,1
4,2019,1,A,골목상권,1000930,언주로81길,CS200001,일반교습학원,527027,53,...,53,53,0,0,0,0,0,53,0,3


In [56]:
df_market_address.head()

,자치구 코드,자치구명,전통시장명,주소명,위도,경도
0,22,은평구,갈현시장,갈현동 424-16,37.623587,126.917385
1,12,동작구,강남시장,상도동 488,37.518749,127.020640
2,1,강남구,강남시장,신사동510-11,37.496746,126.953488
3,1,강남구,강남역 지하도상가,역삼동821-1,37.498595,127.028026
4,15,서초구,강남터미널지하도상점가1구역,서초구 반포동 163,37.500478,126.986060


### 대형마트에 대한 정보도 유동인구, 매출 데이터에 있을까?
- 역에 있는 대형마트는 역에 있는 정보를 사용할 수 있지만, 역 유동인구 = 대형마트 유동인구는 지나친 확대해석이다
- 역에 없는 대형마트는 검색해도 안나온다

In [58]:
df_population[df_population['상권_코드_명'].str.contains('불광')]

,기준 년코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,총_생활인구_수,남성_생활인구_수,여성_생활인구_수,연령대_10_생활인구_수,...,여성연령대_60_이상_토요일시간대_3_생활인구_수,여성연령대_60_이상_토요일시간대_4_생활인구_수,여성연령대_60_이상_토요일시간대_5_생활인구_수,여성연령대_60_이상_토요일시간대_6_생활인구_수,여성연령대_60_이상_일요일시간대_1_생활인구_수,여성연령대_60_이상_일요일시간대_2_생활인구_수,여성연령대_60_이상_일요일시간대_3_생활인구_수,여성연령대_60_이상_일요일시간대_4_생활인구_수,여성연령대_60_이상_일요일시간대_5_생활인구_수,여성연령대_60_이상_일요일시간대_6_생활인구_수
245,2020,4,D,발달상권,1001250,서울 은평구 불광역_3,404002,180468,223534,29165,...,932,858,934,1242,2419,1661,639,577,734,1065
246,2020,4,D,발달상권,1001249,서울 은평구 불광역_2,524727,234024,290703,44675,...,2302,2697,3573,1569,1544,2020,2118,2578,3312,1487
247,2020,4,D,발달상권,1001248,서울 은평구 불광역_1,288839,132906,155933,20523,...,1082,1211,1616,768,854,1056,1010,1165,1497,713
1101,2020,4,A,골목상권,1000394,불광천길,1297473,586444,711030,172406,...,3035,2884,3790,4113,7923,6029,2967,2729,3911,4087
1740,2020,3,D,발달상권,1001250,서울 은평구 불광역_3,417210,185575,231636,30876,...,1022,897,980,1287,2442,1725,683,610,782,1093
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39046,2014,2,D,발달상권,1001248,서울 은평구 불광역_1,557811,260771,297042,30269,...,984,860,1085,279,2,507,679,749,931,194
39898,2014,2,A,골목상권,1000394,불광천길,443065,209485,233583,29359,...,724,603,856,197,0,280,477,493,712,97
40531,2014,1,D,발달상권,1001249,서울 은평구 불광역_2,323435,151877,171558,13881,...,675,608,598,104,0,198,342,402,417,66
40532,2014,1,D,발달상권,1001248,서울 은평구 불광역_1,640888,296320,344566,38251,...,1355,1282,1379,336,0,530,879,1015,1133,234


### 생필품, 매출, 시장이름 데이터에서 시장이름 데이터 들여다보기

In [72]:
# 생필품
necessity_markets = df_necessity[df_necessity['시장유형 구분(시장/마트) 이름']=='전통시장']['시장/마트 이름'].unique()
print(len(necessity_markets), necessity_markets)
print([necessity_market for necessity_market in necessity_markets if not re.search('시장', necessity_market)])

50 ['통인시장' '방배종합시장' '방학동도깨비시장' '장위골목시장' '신창시장' '마포농수산물시장' '동원시장' '광장시장'
 '남성시장' '금남시장' '뚝도시장' '후암시장' '청량리종합시장' '남문시장' '남구로시장' '신영시장' '고척근린시장'
 '용문시장' '대림시장' '영등포전통시장' '망원시장' '영천시장' '인왕시장' '신원시장(신림1동)' '자양골목시장'
 '수유재래시장' '경동시장' '우림시장' '송화시장' '대림중앙시장' '목3동시장' '공릉동 도깨비시장' '돈암제일시장'
 '원당종합시장' '남대문시장' '청담삼익시장' '현대시장' '둔촌역전통시장' '서울중앙시장' '노룬산골목시장' '화곡본동시장'
 '방이시장' '대조시장' '도곡시장' '마천중앙시장' '관악신사시장(신림4동)' '암사종합시장' '상계중앙시장' '숭인시장'
 '방림시장']
[]


In [71]:
# 매출
sales_markets = df_sales[df_sales['상권_구분_코드_명'] == '전통시장']['상권_코드_명'].unique()
print(len(sales_markets), sales_markets)
print([sales_market for sales_market in sales_markets if not re.search('시장', sales_market)])

219 ['영도시장' '암사종합시장' '서울약령시장' '경동시장' '답십리시장' '이경시장' '뚝도시장' '남문시장' '남대문시장'
 '방천골목시장' '이촌종합시장' '신중부시장' '현대시장' '상계중앙시장' '평화시장' '만리시장' '숭인시장' '서울중앙시장'
 '망원시장' '백학시장' '망원월드컵시장' '광장시장' '신흥시장' '장미원골목시장' '약수시장' '신월3동골목시장'
 '성동용답상가시장' '공덕시장' '한아름시장' '노룬산골목시장' '수일시장' '중곡제일시장' '양지골목시장' '신성골목시장'
 '자양골목시장' '능동로골목시장' '동부골목시장' '면목골목시장' '사가정시장' '우이시장' '관악신사시장' '우림골목시장'
 '태능시장' '동원시장' '장위전통시장' '상도4동도깨비골목시장' '솔샘시장' '정릉시장' '상도전통시장' '정릉아리랑시장'
 '길음시장' '인왕시장' '노량진중앙시장' '영천시장' '목2동시장' '번동북부골목시장' '성대시장' '동평화시장' '신영시장'
 '방산종합시장' '신월6동골목시장' '모래내시장' '남성역골목시장' '길동복조리시장' '흑석시장' '전농로터리시장' '동진시장'
 '돈암제일시장' '까치산시장' '합동(새벽)시장' '영진시장A동' '대원종합시장' '새마을시장' '용문전통시장' '화곡본동시장'
 '신원시장' '신림중앙시장' '봉일시장' '신림종합시장' '신수시장' '남성사계시장' '봉천중앙시장' '창동신창시장'
 '명일골목시장' '고덕전통시장' '강남시장_강남' '우림시장' '인헌시장' '금남시장' '성내골목시장' '고척근린시장'
 '신정제일시장' '천호시장' '동묘시장' '신도봉시장' '보광시장' '남부골목시장' '송화골목시장' '미성동도깨비시장'
 '창신골목시장' '회기시장' '답십리현대시장' '백운시장' '상계시장' '화곡중앙골목시장' '본동인정시장' '영등포전통시장'
 '전곡시장' '갈현시장' '포방터시장' '중화제일시장' '가리봉시장' '풍납시장' '오류시장' '마장축산물시장' '목3동시장'
 '은행나무시장

In [79]:
# 시장이름
market_address_markets = df_market_address['전통시장명'].unique()
print(len(market_address_markets), market_address_markets)

327 ['갈현시장' '강남시장' '강남역 지하도상가' '강남터미널지하도상점가1구역' '강남터미널지하도상점가2구역'
 '강남터미널지하도상점가3구역' '강북종합골목시장' '강북종합시장' '개봉중앙시장' '개봉프라자' '개포2동 중심상가'
 '경동광성상가' '경동시장' '경창시장' '고덕전통시장' '고속터미널 지하철역 상점가 ' '고척근린시장' '고척프라자' '공덕시장'
 '공릉동 도깨비시장' '공무원연금매장 상록스토아 개포점' '공항시장' '관악신사시장(신림4동)' '관악종합시장' '광성시장'
 '광장골목시장' '광장시장' '광희시장' '구로시장' '구의시장' '굽은다리골목시장' '금남시장' '금천교시장' '길동골목시장'
 '길음시장' '까치산시장' '깡통시장(제기상가번영회)' '낙성대 시장(봉천7동)' '낙원상가' '낙원지하시장(대일상가)'
 '남구로시장(구 구로자율시장)' '남대문로 지하상가' '남대문시장' '남문시장' '남부골목시장' '남부종합시장' '남부화곡시장'
 '남서울상가' '남평화시장' '노룬산골목시장' '노룬산시장' '녹번시장' '논현종합시장' '능동로골목시장' '답십리시장'
 '답십리현대시장' '대림골목시장' '대림시장' '대림중앙시장' '대명시장' '대명시장 및 주변상점가' '대신시장' '대원종합시장'
 '대조시장' '도림시장' '도선동상점가' '도화동 상점가' '돈암제일시장' '동남상가' '동대문문구완구종합시장' '동대문상가A동'
 '동대문상가B동' '동대문상가C동' '동대문종합시장' '동대문종합시장D동상가' '동대문지하쇼핑센터' '동문시장' '동부골목시장'
 '동부시장' '동부청과시장' '동북시장' '동서시장' '동서울시장' '동성타워프라자' '동소문 스카이상가' '동원골목시장'
 '동원시장' '동진시장' '동평화시장' '동화시장' '두암시장' '둔촌역전통시장' '등마루시장' '뚝도시장' '로데오거리상점가'
 '로타리상가' '마장축산물시장' '마천시장' '마천중앙시장' '마포농수산물시장' '마포시장' '만리시장' '망원동월드컵시장'
 '망

### 생필품 데이터의 시장주소가 시장주소 데이터의 시장주소에 매핑되는지 확인

In [59]:
df_market_address[df_market_address['전통시장명'].str.contains('남구로')]

,자치구 코드,자치구명,전통시장명,주소명,위도,경도
41,7,구로구,남구로시장(구 구로자율시장),구로동 730-27,37.490481,126.885048


In [154]:

none_necessity_markets = set(necessity_markets) - set(market_address_markets)
print(len(necessity_markets), len(none_necessity_markets), none_necessity_markets)

50 7 {'남구로시장', '방림시장', '신창시장', '송화시장', '남성시장', '방배종합시장', '도곡시장'}


In [157]:
market_market_address_to_necessity = {}
none_necessity_markets2 = set(none_necessity_markets)


# 창동신창시장은 검색결과 신창시장 맞음
# 보통 남성시장이라고 검색했을 때 이수역의 남성사계시장 의미
pre_market = re.compile(r'(중앙)?(재래)?(골목)?(종합)?시장[가-힣()1-9 ]*')
for none_necessity_market in none_necessity_markets:
    r_market = re.compile(pre_market.sub('', none_necessity_market))
    for market_address_market in market_address_markets:
        if r_market.search(market_address_market):
            print(none_necessity_market, '/', market_address_market)
            market_market_address_to_necessity[market_address_market] = none_necessity_market
            none_necessity_markets2.remove(none_necessity_market)
            break
            
print(market_market_address_to_necessity)
print(len(none_necessity_markets2), none_necessity_markets2)

남구로시장 / 남구로시장(구 구로자율시장)
신창시장 / 창동신창시장
송화시장 / 송화골목시장
{'남구로시장(구 구로자율시장)': '남구로시장', '창동신창시장': '신창시장', '송화골목시장': '송화시장'}
4 {'남성시장', '방림시장', '방배종합시장', '도곡시장'}


### 생필품 데이터의 시장주소가 매출 데이터의 시장주소에 매핑되는지 확인

In [95]:
none_necessity_markets = set(necessity_markets) - set(sales_markets)
print(len(necessity_markets), len(none_necessity_markets), none_necessity_markets)

50 14 {'관악신사시장(신림4동)', '방림시장', '대림시장', '원당종합시장', '마천중앙시장', '신원시장(신림1동)', '송화시장', '수유재래시장', '신창시장', '장위골목시장', '남성시장', '용문시장', '공릉동 도깨비시장', '방배종합시장'}


In [152]:
market_sales_to_necessity = {}
none_necessity_markets2 = set(none_necessity_markets)

# 마천시장 = 마천중앙시장
# 수유전통시장 = 수유재래시장(카카오맵 위치상 거의 같음)
# 장위골목시장 ? 장위전통시장(골목 검색결과 없음)
# 남성시장 != 남성역골목시장(남성사계시장(옛이름 남성시장), 남성역골목시장은 각각 이수, 남성역에 위치)
# 용문시장 ? 용문전통시장(용문전통시장 검색결과 없음)
pre_market = re.compile(r'(중앙)?(재래)?(골목)?(종합)?시장[가-힣()1-9 ]*')
# pre_market = re.compile(r'[()]')
for none_necessity_market in none_necessity_markets:
#     print(pre_market.sub('', none_necessity_market))
    r_market = re.compile(pre_market.sub('', none_necessity_market))
    for sales_market in sales_markets:
        if r_market.search(sales_market):
            print(none_necessity_market, '/', sales_market)
            market_sales_to_necessity[sales_market] = none_necessity_market
            none_necessity_markets2.remove(none_necessity_market)
            break
            
print(market_sales_to_necessity)
print(len(none_necessity_markets2), none_necessity_markets2)

관악신사시장(신림4동) / 관악신사시장
대림시장 / 대림골목시장
마천중앙시장 / 마천시장
신원시장(신림1동) / 신원시장
송화시장 / 송화골목시장
수유재래시장 / 수유전통시장
신창시장 / 창동신창시장
장위골목시장 / 장위전통시장
남성시장 / 남성역골목시장
용문시장 / 용문전통시장
{'관악신사시장': '관악신사시장(신림4동)', '대림골목시장': '대림시장', '마천시장': '마천중앙시장', '신원시장': '신원시장(신림1동)', '송화골목시장': '송화시장', '수유전통시장': '수유재래시장', '창동신창시장': '신창시장', '장위전통시장': '장위골목시장', '남성역골목시장': '남성시장', '용문전통시장': '용문시장'}
4 {'방림시장', '원당종합시장', '공릉동 도깨비시장', '방배종합시장'}


### 생필품, 매출, 시장이름 데이터에서 시장이름이 같으면 코드도 같은가?

In [190]:
df_necessity.groupby(['시장/마트 이름', '년도-월', '품목 번호', '실판매규격', '비고']).count()[['시장/마트 번호']]

시장/마트 번호
시장/마트 이름    년도-월    품목 번호 실판매규격     비고               
2001아울렛 불광점 2020-01 23    100g      여주              3
                                    용인              1
                                    이천              2
                    25    1개        제주              1
                    27    1개        나주              1
...                                               ...
후암시장        2020-12 312   1개        청원생명            1
                    315   1마리 45cm  러시아산 냉동         1
                          1마리 45cm  러시아산 냉동         5
                    320   30개       국내산 참사랑         3
                                    참사랑             3

[44531 rows x 1 columns]

In [180]:
df_necessity.groupby(['시장/마트 번호', '년도-월', '품목 번호', '실판매규격']).count()['시장/마트 이름'].unique()

array([ 8,  7,  1,  6,  5,  2,  9,  4,  3, 10, 11], dtype=int64)

In [196]:
print(len(df_necessity['시장/마트 번호'].unique()))
print(len(df_necessity['시장/마트 이름'].unique()))
# df_necessity[df_necessity['일련번호'] == 1519524]
for num in df_necessity['시장/마트 번호'].unique():
    df_nessity_market = df_necessity[df_necessity['시장/마트 번호'] == num]
    print(df_nessity_market['시장/마트 이름'].unique())

102
103
['통인시장']
['방배종합시장']
['NC백화점 불광점' '2001아울렛 불광점']
['홈플러스 목동점']
['홈플러스 독산점']
['신세계백화점']
['방학동도깨비시장']
['NC백화점 신구로점']
['농협하나로마트 신촌점']
['롯데마트 서울역점']
['장위골목시장']
['이마트 미아점']
['신창시장']
['농협 하나로마트 용산점']
['이마트 은평점']
['마포농수산물시장']
['이마트 신도림점']
['동원시장']
['광장시장']
['이마트 창동점']
['남성시장']
['금남시장']
['롯데백화점']
['뚝도시장']
['롯데백화점 관악점']
['홈플러스 중계점']
['후암시장']
['청량리종합시장']
['남문시장']
['홈플러스 월드컵점']
['남구로시장']
['신영시장']
['태평백화점']
['이마트 자양점']
['고척근린시장']
['롯데마트 강변점']
['용문시장']
['홈플러스 방학점']
['이마트 청계점']
['대림시장']
['영등포전통시장']
['망원시장']
['영천시장']
['이마트 왕십리점']
['이마트 역삼점']
['현대백화점 미아점']
['인왕시장']
['이마트 성수점']
['홈플러스 동대문점']
['신원시장(신림1동)']
['이마트 가양점']
['자양골목시장']
['수유재래시장']
['경동시장']
['롯데백화점 미아점']
['우림시장']
['롯데백화점 강남점']
['홈플러스 등촌점']
['롯데백화점 청량리점']
['송화시장']
['롯데백화점 노원점']
['대림중앙시장']
['현대백화점 신촌점']
['홈플러스 면목점']
['목3동시장']
['이마트 용산점']
['공릉동 도깨비시장']
['돈암제일시장']
['원당종합시장']
['홈플러스 시흥점']
['남대문시장']
['롯데백화점 영등포점']
['청담삼익시장']
['현대시장']
['신세계백화점 강남점']
['홈플러스 영등포점']
['이마트 여의도점']
['롯데슈퍼']
['둔촌역전통시장']
['하나로클럽 미아점']
['이마트 상봉점']
['홈플러스 강동점']
['이마트 명일점'

In [197]:
print(len(df_necessity['시장/마트 번호'].unique()))
print(len(df_necessity['시장/마트 이름'].unique()))
# df_necessity[df_necessity['일련번호'] == 1519524]
for name in df_necessity['시장/마트 이름'].unique():
    df_nessity_market = df_necessity[df_necessity['시장/마트 이름'] == name]
    print(df_nessity_market['시장/마트 번호'].unique())

102
103
[1]
[228]
[51]
[223]
[222]
[2]
[25]
[227]
[224]
[8]
[14]
[15]
[26]
[63]
[53]
[121]
[110]
[91]
[55]
[27]
[127]
[72]
[56]
[73]
[135]
[100]
[62]
[84]
[115]
[123]
[108]
[103]
[129]
[80]
[109]
[82]
[10]
[28]
[60]
[49]
[20]
[120]
[32]
[74]
[44]
[16]
[31]
[75]
[86]
[134]
[42]
[78]
[66]
[85]
[68]
[90]
[45]
[40]
[88]
[38]
[98]
[19]
[33]
[94]
[102]
[6]
[96]
[13]
[133]
[116]
[11]
[130]
[215]
[114]
[218]
[21]
[22]
[220]
[205]
[206]
[92]
[154]
[153]
[199]
[212]
[213]
[147]
[214]
[146]
[219]
[216]
[200]
[136]
[145]
[207]
[151]
[210]
[221]
[148]
[140]
[226]
[51]
[217]


In [165]:
# 1만 있으면 같은 것
df_sales.groupby(['상권_코드_명', '기준_년_코드', '기준_분기_코드', '서비스_업종_코드']).count()['상권_코드'].unique()

array([1], dtype=int64)

In [205]:
df_market_address.groupby('전통시장명').count()['주소명']

전통시장명
갈현시장              1
강남시장              2
강남역 지하도상가         1
강남터미널지하도상점가1구역    1
강남터미널지하도상점가2구역    1
                 ..
화곡중앙시장            1
화양제일골목시장          1
회기시장              1
회현지하도상가           1
후암시장              1
Name: 주소명, Length: 327, dtype: int64

In [206]:
# 강남시장의 경우 동일이름 다른지역에 2개 있음
# 하지만 생필품 데이터에 강남시장이 없어 넘어가도 됨
df_market_address[df_market_address['전통시장명'] == '강남시장']

,자치구 코드,자치구명,전통시장명,주소명,위도,경도
1,12,동작구,강남시장,상도동 488,37.518749,127.020640
2,1,강남구,강남시장,신사동510-11,37.496746,126.953488


### 매출 데이터에서 업종이름이 같으면 코드도 같은가?

In [176]:
# 1만 있으면 같은 것
df_sales.groupby(['서비스_업종_코드_명', '기준_년_코드', '기준_분기_코드', '상권_코드']).count()['서비스_업종_코드'].unique()

array([1], dtype=int64)